# Initialization

In [1]:
from cellpose import models, io
from cellpose.io import *
from collections import defaultdict
import geopandas
import glob
import imagej
from jpype import JArray, JInt
import matplotlib.pyplot as plt
import multiprocessing as mp
import numpy as np
import os
import re
import pandas
from pandas import DataFrame
from pathlib import Path
import scyjava
import seaborn
import shutil
import tkinter as tk
from tkinter import filedialog
from PIL import Image
import sys
import os
from zipfile import ZipFile
import csv
import random
import math
from math import isnan 

In [2]:
# Path to your local fiji install and amount of RAM you want to allocate to the java session
fiji_path = '/home/saka/sw/local/fiji/2023'
memory = 30

scyjava.config.add_option('-Xmx'+str(memory)+'g')
scyjava.config.add_option('-Xss1024K')
ij = imagej.init(fiji_path, mode='interactive')
ij.ui().showUI()
ij.getVersion()

# this part is the import of the fiji function into the pyimagej environment
showPolygonRoi = scyjava.jimport('ij.gui.PolygonRoi')
Overlay = scyjava.jimport('ij.gui.Overlay')
Regions = scyjava.jimport('net.imglib2.roi.Regions')
LabelRegions = scyjava.jimport('net.imglib2.roi.labeling.LabelRegions')
ZProjector = scyjava.jimport('ij.plugin.ZProjector')()
Duplicator = scyjava.jimport('ij.plugin.Duplicator')()
TrackMate = scyjava.jimport('fiji.plugin.trackmate.TrackMate')
ov = Overlay()
Model =  scyjava.jimport('fiji.plugin.trackmate.Model')
Settings= scyjava.jimport('fiji.plugin.trackmate.Settings')
Settings= scyjava.jimport('fiji.plugin.trackmate.Settings')
TrackMate = scyjava.jimport('fiji.plugin.trackmate.TrackMate')
Logger= scyjava.jimport('fiji.plugin.trackmate.Logger')
DetectorKeys= scyjava.jimport('fiji.plugin.trackmate.detection.DetectorKeys') 
ExportTracksToXML= scyjava.jimport('fiji.plugin.trackmate.action.ExportTracksToXML') 
TmXmlWriter= scyjava.jimport('fiji.plugin.trackmate.io.TmXmlWriter')
TmXmlReader = scyjava.jimport('fiji.plugin.trackmate.io.TmXmlReader')
LogRecorder = scyjava.jimport('fiji.plugin.trackmate.util.LogRecorder')
SparseLAPTrackerFactory= scyjava.jimport('fiji.plugin.trackmate.tracking.jaqaman.SparseLAPTrackerFactory')
TMUtils = scyjava.jimport('fiji.plugin.trackmate.util.TMUtils')
HyperStackDisplayer = scyjava.jimport('fiji.plugin.trackmate.visualization.hyperstack.HyperStackDisplayer')
SelectionModel = scyjava.jimport('fiji.plugin.trackmate.SelectionModel')
CellposeDetectorFactory = scyjava.jimport('fiji.plugin.trackmate.cellpose.CellposeDetectorFactory')
FeatureFilter = scyjava.jimport('fiji.plugin.trackmate.features.FeatureFilter')
DisplaySetting = scyjava.jimport('fiji.plugin.trackmate.gui.displaysettings.DisplaySettings')
DisplaySettingsIO = scyjava.jimport('fiji.plugin.trackmate.gui.displaysettings.DisplaySettingsIO')
CaptureOverlayAction = scyjava.jimport('fiji.plugin.trackmate.action.CaptureOverlayAction')
PretrainedModel= scyjava.jimport('fiji.plugin.trackmate.cellpose.CellposeSettings.PretrainedModel')
ThresholdDetectorFactory= scyjava.jimport('fiji.plugin.trackmate.detection.ThresholdDetectorFactory')
TrackScheme = scyjava.jimport('fiji.plugin.trackmate.visualization.trackscheme.TrackScheme')
TrackTableView = scyjava.jimport('fiji.plugin.trackmate.visualization.table.TrackTableView')
AllSpotsTableView = scyjava.jimport('fiji.plugin.trackmate.visualization.table.AllSpotsTableView')
PretrainedModel = scyjava.jimport('fiji.plugin.trackmate.cellpose.CellposeSettings.PretrainedModel')
Logger= scyjava.jimport('fiji.plugin.trackmate.Logger')
DetectorKeys= scyjava.jimport('fiji.plugin.trackmate.detection.DetectorKeys') 
ExportTracksToXML= scyjava.jimport('fiji.plugin.trackmate.action.ExportTracksToXML') 
LogRecorder = scyjava.jimport('fiji.plugin.trackmate.util.LogRecorder')
SparseLAPTrackerFactory= scyjava.jimport('fiji.plugin.trackmate.tracking.jaqaman.SparseLAPTrackerFactory')
TMUtils = scyjava.jimport('fiji.plugin.trackmate.util.TMUtils')
HyperStackDisplayer = scyjava.jimport('fiji.plugin.trackmate.visualization.hyperstack.HyperStackDisplayer')
SelectionModel = scyjava.jimport('fiji.plugin.trackmate.SelectionModel')
CellposeDetectorFactory = scyjava.jimport('fiji.plugin.trackmate.cellpose.CellposeDetectorFactory')
FeatureFilter = scyjava.jimport('fiji.plugin.trackmate.features.FeatureFilter')
DisplaySetting = scyjava.jimport('fiji.plugin.trackmate.gui.displaysettings.DisplaySettings')
DisplaySettingsIO = scyjava.jimport('fiji.plugin.trackmate.gui.displaysettings.DisplaySettingsIO')
CaptureOverlayAction = scyjava.jimport('fiji.plugin.trackmate.action.CaptureOverlayAction')
PretrainedModel= scyjava.jimport('fiji.plugin.trackmate.cellpose.CellposeSettings.PretrainedModel')
ThresholdDetectorFactory= scyjava.jimport('fiji.plugin.trackmate.detection.ThresholdDetectorFactory')
TrackScheme = scyjava.jimport('fiji.plugin.trackmate.visualization.trackscheme.TrackScheme')
TrackTableView = scyjava.jimport('fiji.plugin.trackmate.visualization.table.TrackTableView')
AllSpotsTableView = scyjava.jimport('fiji.plugin.trackmate.visualization.table.AllSpotsTableView')
PretrainedModel = scyjava.jimport('fiji.plugin.trackmate.cellpose.CellposeSettings.PretrainedModel')
IJRoiExporter = scyjava.jimport('fiji.plugin.trackmate.action.IJRoiExporter')
rm = ij.RoiManager.getRoiManager()

# Functions

In [3]:
def open_raw_image(raw_image_path):
    '''
    This function loads the raw image file into ImageJ from the path specified. The raw image path should be a .czi file.
    '''
    raw_image = ij.io().open(raw_image_path)
    return raw_image

def get_z_stack(directory_path, imp, method, dimensions):
    zproject = ZProjector.run(imp, method)
    zproject.setDimensions(dimensions[0], dimensions[1], dimensions[2])
    new_channels = directory_path + method[0:len(method)-4] + "_channels.tif"
    ij.IJ.saveAs(zproject, "Tiff", ij.py.to_java(new_channels))
    return new_channels

def process_images(directory_path, raw_image, method='max all', wanted_z=2, dimensions=[4,1,121], get_sum_channels=True):
    '''
    This function retrieves a multi-channel, single-slice T-stack image at the specified z-value and saves it as channels.tif.
    It also creates a z-projection from the raw image using the method specified and saves it as [method]_channels.tif. The default method is a "max" z-projection.
    '''
    imp = ij.py.to_imageplus(raw_image)
    channels = raw_image[:, :, :, wanted_z, :]
    final_image = ij.py.to_imageplus(channels)
    final_image.setDimensions(dimensions[0], dimensions[1], dimensions[2])
    channels_image = directory_path + "channels.tif"
    saved = ij.IJ.save(final_image, ij.py.to_java(channels_image))
    max_channels = get_z_stack(directory_path, imp, method, dimensions)
    if get_sum_channels:
        get_z_stack(directory_path, imp, 'sum all', dimensions)
    return channels_image, max_channels

def process_channels(directory_path, raw_image, channel_dict={'LC3':0,'LV':2,'bact':1}, dimensions=[1, 11, 121]):
    '''
    This function isolates and saves single-channel videos from the raw image for each desired channel specified in channel_dict. 
    Note on channel_dict: channel_dict is a dictionary where each key is a string that represents a channel name and the corresponding values are integers equal to the corresponding channel number in the multi-channel video. 
    '''
    for c in channel_dict:
        channel = raw_image[:, :, channel_dict[c], :, :]
        c_image = ij.py.to_imageplus(channel)
        c_image.setDimensions(dimensions[0], dimensions[1], dimensions[2])
        result_path = f"{directory_path}/channel_{c}.tif"
        ij.IJ.saveAs(c_image, "Tiff", ij.py.to_java(result_path))

def track_cells(channels_image, directory_path, dsettings, tsettings, distance=2048, known=202.93, unit="µm"):
    '''
    This function leverages ImageJ's Trackmate plugin to track the cells in channels.tif. It uses a custom-trained Cellpose model for segmentation (with detection parameters specified in dsettings), and the LAP Tracker algorithm for tracking (with tracking parameters specified in tsettings).
    The function saves the Trackmate XML model as well as a CSV file of the tracks as output. The function also takes as input the scale of the image.
    '''
    # Open Image
    imp = ij.IJ.openImage(channels_image)
    imp.show()
    ij.py.run_macro(f'run("Set Scale...", "distance={distance} known={known} unit={unit}");')    
    # Create Model
    model = Model()
    settings = Settings(imp)   
    # Detector
    settings.detectorFactory = CellposeDetectorFactory()
    for parameter, value in dsettings.items():
        settings.detectorSettings[parameter] = value
    # Tracker
    settings.trackerFactory = SparseLAPTrackerFactory()
    settings.trackerSettings = settings.trackerFactory.getDefaultSettings()
    for parameter, value in tsettings.items():
        settings.trackerSettings[parameter] = value
    # Execute Tracking
    trackmate = TrackMate(model, settings)
    ok = trackmate.checkInput()
    if not ok:
        sys.exit(str(trackmate.getErrorMessage()))
    ok = trackmate.process()
    if not ok:
        sys.exit(str(trackmate.getErrorMessage()))
    selectionModel = SelectionModel(model)  
    # Display
    ds = DisplaySettingsIO.readUserDefault()
    displayer = HyperStackDisplayer(model, selectionModel, imp, ds)
    displayer.render()
    displayer.refresh()
    trackscheme = TrackScheme(model, selectionModel, ds)
    trackscheme.render()
    # Save Data
    outFile = Path(directory_path+"exportModel.xml")
    writer = TmXmlWriter(outFile)
    writer.appendModel(model)
    writer.appendSettings(settings)
    writer.writeToFile()
    csvFileTracks = Path(directory_path+"exportTracks.csv")
    trackTableView = TrackTableView(model, selectionModel, ds)
    trackTableView.getSpotTable().exportToCsv(csvFileTracks)
    return csvFileTracks, model, imp

def sort_trackmate_export(tm_tracks_csv, directory_path):
    '''
    This function sorts the exported CSV file from Trackmate so that the cells are ordered by track ID and frame number. It saves the sorted file to the directory path.
    '''
    df = pandas.read_csv(tm_tracks_csv)
    df.drop([0, 1, 2], axis=0, inplace=True)
    df = df.reset_index(drop=True)
    for index, row in df.iterrows():
        df.at[index, 'TRACK_ID'] = int(row['TRACK_ID'])
        df.at[index, 'FRAME'] = int(row['FRAME'])
    df.sort_values(by=['TRACK_ID', 'FRAME'], inplace=True, ignore_index=True)
    df.to_csv(directory_path+"exportTracks_edited.csv", index=False)
    output = directory_path+"exportTracks_edited.csv"
    return output

def get_cell_rois(directory_path, model, imp):
    '''
    This function exports the cell ROIs from Trackmate and saves them in a zip file. 
    '''
    model.setLogger(Logger.IJ_LOGGER)
    logger = Logger.IJ_LOGGER
    exporter = IJRoiExporter(imp, logger)
    exporter.export(model.getSpots().iterable(True))
    rm.save(directory_path+"RoiSet_cells.zip");
    rm.runCommand(imp,"Delete");
    with ZipFile(directory_path+"RoiSet_cells.zip", 'r') as zip: 
        zip.extractall(path=directory_path+"cell_rois")
    output = directory_path+"cell_rois/"
    return output

def create_tracks_csv(sorted_tm_tracks_csv, directory_path):
    '''
    This function creates a CSV file (da_Tracks.csv) where each row is a separate track and lists the ROI Manager indices that correspond to cell ROIs in that track.
    '''
    data = pandas.read_csv(sorted_tm_tracks_csv)
    file = open(directory_path+'da_Tracks.csv', 'w', newline='')
    csv_writer = csv.writer(file)
    current_track = 0
    first = True
    string = ''
    for index, row in data.iterrows():
        if int(row['TRACK_ID']) == current_track:
            if first:
                string = f'{index+1}'
                first = False
            else:
                string = string + ", " + f'{index+1}'
        else:
            csv_writer.writerow([string])
            string = f'{index+1}'
            current_track += 1  
    file.close()
    output = directory_path+"da_Tracks.csv"
    return output

def add_rois(directory_path, sorted_tm_tracks_csv, roiset):
    '''
    This function adds cell ROIs into the ImageJ ROI Manager *in order*. If the sorted ROI zip file already exists, then it directly loads it into the ROI Manager. Otherwise, it references sorted_tracks_csv and saves the sorted zip file at the end.
    It is imperative to run this function to load the ROI Manager (as opposed to dragging and dropping the RoiSet into ImageJ GUI) in order to preserve the order of the ROIS and ensure consistency with ROI labels given in da_Tracks.csv.
    '''
    rm.runCommand("Reset")
    if os.path.exists(directory_path+"sorted_RoiSet_cells.zip"):
        rm.open(directory_path+"sorted_RoiSet_cells.zip")
    else:
        df = pandas.read_csv(sorted_tm_tracks_csv)
        for index, row in df.iterrows():
            file = row['LABEL'] + '.roi'
            rm.open(roiset+file)
        rm.save(directory_path+"sorted_RoiSet_cells.zip")

def apply_cell_overlays(directory_path, max_channels, tracks_csv, sorted_tm_tracks_csv, roiset):
    '''
    This function allows for visual quality-control of the cell-tracking step by displaying the max z-projection video and applying a colored overlay to the cells in each track. 
    It is a useful aid while selecting cells of interest since cells of the same track remain the same color from frame to frame.
    '''
    imp = ij.IJ.openImage(max_channels)
    imp.show()
    rm.runCommand("Reset")
    add_rois(directory_path, sorted_tm_tracks_csv, roiset)
    data_info = {}
    for element in range(len(imp.dims)):
        name = imp.dims[element]
        data_info[name] = imp.shape[element]
    num_channel = data_info['C'] + 1
    for n in range (1, num_channel):
        imp.setC(n)
        ij.IJ.resetMinAndMax(imp)
    imp.setC(1)
    ij.IJ.run(imp, "Green", "");
    imp.setC(2)
    ij.IJ.run(imp, "Red", "");
    df2 = pandas.read_csv(tracks_csv, header=None)
    colors = ["black", "blue", "cyan", "green", "magenta", "orange", "red", "white", "yellow"]
    for index, row in df2.iterrows():
        l = row[0].split(", ")
        random_color = random.choice(colors)
        for cell in l:
            cell_index = int(cell)
            roi = rm.select(cell_index-1)
            overlay_command = f"Overlay.addSelection('{random_color}', 5);"
            ij.py.run_macro(overlay_command)

def isolate_single_cell(directory_path, channel_dict, tracks_csv, sorted_tm_tracks_csv, roiset, cells_desired=[]):
    '''
    This function isolates and saves single-cell, single-channel videos for each cell track specified in cells_desired.
    Videos with the "bact" signal are saved in directory_path/bact/ and videos for all other channels in channel_dict are saved in directory_path/other/.
    Notes on cells_desired: 
        To duplicate all of the segmented cells in the video, input an empty list for cells_desired.
            Ex. cells_desired = [] will return single-cell videos for every row in da_Tracks.
        Otherwise, cells_desired should be a list where each item is either a) an integer corresponding to the row number of the desired track in da_Tracks.csv or b) a list of integers corresponding to row numbers of cell tracks that should be concatenated (useful for fragmented tracks)
            Ex. cells_desired = [1, 3, 7, [9,10,11]] will return single-cell videos for row 1, row 3, and row 7 of da_Tracks, as well as a single-cell video that merges row 9, row 10, and row 11's cell tracks.
    '''
    bact = directory_path+"bact/"
    other = directory_path+"other_signals/"
    if not os.path.exists(bact):
        os.makedirs(bact)
    if not os.path.exists(other):
        os.makedirs(other)
    if os.path.exists(directory_path+"da_Frames.csv"):
        f=pandas.read_csv(directory_path+"da_Frames.csv")
        frames_dict = pandas.Series(f['Total Frames'].values, index=f['Cell Number']).to_dict()
    else:
        frames_dict = {}
    rm.runCommand("Reset")
    add_rois(directory_path, sorted_tm_tracks_csv, roiset)
    signal_list = list(channel_dict)
    df2 = pandas.read_csv(tracks_csv, header=None)
    if len(cells_desired) == 0:
        for n in range(1,len(df2)+1):
            cells_desired.append(n)
    for signal in signal_list:
        image_path = Path(f"{directory_path}/channel_{signal}.tif").as_posix()
        imp = ij.IJ.openImage(image_path)
        imp.show()
        w = scyjava.jimport('ij.WindowManager')
        df = pandas.read_csv(tracks_csv, header=None)
        for cell_desired in cells_desired:
            nums = []
            if isinstance(cell_desired, int):
                row = df.iloc[cell_desired-1]
                nums = row[0].split(", ")
            elif isinstance(cell_desired, list):
                for c in cell_desired:
                    row = df.iloc[c-1]
                    nums += row[0].split(", ")
            arr = []
            for x in nums:
                arr.append(int(x))
            for i in range(len(arr)):
                cell = arr[i]
                roi = rm.getRoi(cell-1)
                roi2 = rm.select(cell-1)
                frame = roi.getTPosition()
                if i == 0:
                    ij.IJ.run("Duplicate...", "title=Stack_reg_image duplicate frames="+str(frame))
                    ij.IJ.run("Clear Outside", "stack")
                    ij.IJ.selectWindow(imp.getTitle())
                elif i > 0:
                    ij.IJ.run("Duplicate...", "title=channels duplicate frames="+str(frame))
                    ij.IJ.run("Clear Outside", "stack")
                    ij.IJ.run("Concatenate...", "title=Stack_reg_image open image1=Stack_reg_image image2=channels image3=[-- None --]")
                    ij.IJ.selectWindow(imp.getTitle())
            new_vid =  w.getImage("Stack_reg_image")
            if signal == "bact":
                ij.IJ.save(new_vid, bact+"cell_"+str(cell_desired)+".tif")
            else:
                ij.IJ.save(new_vid, other+signal+"_cell_"+str(cell_desired)+".tif")
            w.getImage("Stack_reg_image").close()
            ij.IJ.selectWindow(imp.getTitle())
            if str(cell_desired) not in frames_dict:
                frames_dict[f"{cell_desired}"] = len(arr)
        ij.py.run_macro('close("*")')
    data = pandas.DataFrame(list(frames_dict.items()))
    data.columns = ['Cell Number', 'Total Frames']
    data.to_csv(directory_path+"da_Frames.csv", index=False)
    return bact, other

def get_bacteria_rois(bact, directory_path, threshold_low, threshold_high, threshold_size, distance, known, unit):
    '''
    This function processes every single-cell video in the input folder bact. It uses thresholding in ImageJ to detect bacterial ROIs within the video and saves them in a zip file with the cell name. 
    The function also creates and saves a CSV file of all of the bacterial ROIs for each cell video.
    '''
    dict = {}
    file_pattern = os.path.join(bact, "*.tif")
    file_list = glob.glob(file_pattern)
    for file_path in file_list:
        if not file_path.endswith("overlay.tif"):
            rm.runCommand("Reset")
            f_name = os.path.basename(file_path)
            f_name = os.path.splitext(f_name)[0]
            imp = ij.IJ.openImage(file_path)
            print(f"Opening {file_path}")
            data_info = {}
            for element in range(len(imp.dims)):
                name = imp.dims[element]
                data_info[name] = imp.shape[element]
            num_frames = data_info['T']
            dict.update({f_name.split("_")[1] : num_frames})
            ij.IJ.run(imp, "Smooth", "stack")
            ij.IJ.run(imp, "Smooth", "stack")
            imp.show()
            ij.py.run_macro(f'run("Set Scale...", "distance={distance} known={known} unit={unit}");')    
            ij.IJ.run("Set Measurements...", f'area centroid stack decimal=2')
            ij.IJ.setAutoThreshold(imp, "Default dark")
            ij.IJ.setRawThreshold(imp, threshold_low, threshold_high)
            ij.IJ.run(imp, "Convert to Mask", "background=Dark black");
            ij.IJ.run(imp, "Analyze Particles...", f"size={threshold_size}-Infinity display add stack");
            ij.IJ.run(imp, "Measure", "");
            rm.save(f"{bact}" + f"{f_name}.zip")
            output_path = Path(f"{bact}{f_name}.csv").as_posix()
            ij.IJ.saveAs("Results", output_path)
            ij.IJ.run("Clear Results")
            ij.py.run_macro('close("*");')
        rm.runCommand("Reset")
        #ij.IJ.selectWindow("Results")
        #ij.IJ.run("Close")
        data = pandas.DataFrame(list(dict.items()))
        data.columns = ['Cell Number', 'Total Frames']
        data.to_csv(directory_path+"da_Frames.csv", index=False)

def track_bacteria(bact, directory_path, dsettings, quality_filter, tsettings, distance, known, unit, display=False):
    '''
    This function leverages ImageJ's Trackmate plugin to track the bacteria within each single-cell video provided in the input folder bact. 
    It uses the Tresholding Detector for segmentation (with detection parameters specified in dsettings), and the LAP Tracker algorithm for tracking (with tracking parameters specified in tsettings). The function also takes as input the desired quality filter on the detected spots.
    The function saves the Trackmate XML model as well as a CSV file of the tracks for each cell in bact/Output/.
    '''
    out = directory_path+"Single_Cell_Tracking_Output/"
    if not os.path.exists(out):
        os.makedirs(out)
    for image in os.listdir(bact):
        if not image.endswith("overlay.tif"):
            if (image[len(image)-4:] == ".tif"):
                
                # Open Image
                imp = ij.IJ.openImage(bact + image)
                imp.show()
                ij.py.run_macro(f'run("Set Scale...", "distance={distance} known={known} unit={unit}");')
                ij.IJ.run("Smooth", "stack")
                ij.IJ.run("Smooth", "stack")
                # Create Model
                model = Model()
                settings = Settings(imp)
                # Detector
                settings.detectorFactory = ThresholdDetectorFactory()
                for parameter, value in dsettings.items():
                    settings.detectorSettings[parameter] = value
                filter1 = FeatureFilter('QUALITY', quality_filter, True)
                settings.addSpotFilter(filter1)
                print(settings.detectorSettings)
                # Tracker
                settings.trackerFactory = SparseLAPTrackerFactory()
                settings.trackerSettings = settings.trackerFactory.getDefaultSettings()
                for parameter, value in tsettings.items():
                    settings.trackerSettings[parameter] = value
                # Execute Tracking
                trackmate = TrackMate(model, settings)
                ok = trackmate.checkInput()
                if not ok:
                    sys.exit(str(trackmate.getErrorMessage()))
                ok = trackmate.process()
                if not ok:
                    sys.exit(str(trackmate.getErrorMessage()))
                selectionModel = SelectionModel(model)
                # Display
                ds = DisplaySettingsIO.readUserDefault()
                if display:
                    displayer = HyperStackDisplayer(model, selectionModel, imp, ds)
                    displayer.render()
                    displayer.refresh()
                    trackscheme = TrackScheme(model, selectionModel, ds)
                    trackscheme.render()
                # Save Data
                outFile = Path(out+image+"_exportModel.xml")
                writer = TmXmlWriter(outFile)
                writer.appendModel(model)
                writer.appendSettings(settings)
                writer.writeToFile()
                csvFileTracks = Path(out+image+"_exportTracks.csv")
                trackTableView = TrackTableView(model, selectionModel, ds)
                trackTableView.getSpotTable().exportToCsv(csvFileTracks)
        ij.py.run_macro('close("*")')

def xref_locations(first, second, first_x='POSITION_X', first_y='POSITION_Y', first_z='POSITION_Z',
                   second_x='X', second_y='Y', second_z='Slice',
                   max_dist=2, verbose=False):
    pairwise_elements = pandas.DataFrame()
    first_measurements = pandas.read_csv(first)
    first_measurements = first_measurements.drop([0,1,2])
    second_measurements = pandas.read_csv(second)
    first_gdf = geopandas.GeoDataFrame(first_measurements, geometry=geopandas.points_from_xy(first_measurements[first_x],
                                          first_measurements[first_y],
                                          first_measurements[first_z]))
    second_gdf = geopandas.GeoDataFrame(second_measurements, geometry=geopandas.points_from_xy(second_measurements[second_x],
                                          second_measurements[second_y],
                                          second_measurements[second_z]))
    ti_rows = first_gdf.shape[0]
    tj_rows = second_gdf.shape[0]
    for ti_row in range(0, ti_rows):
        if verbose:
            print(f"On row: {ti_row}")
        ti_element = first_gdf.iloc[[ti_row, ]]
        # Get the frame of the current element
        ti_frame = int(ti_element['FRAME'].values[0])  # Convert to integer values in 'FRAME' column
        # Filter second_gdf to only include elements with the same frame
        same_frame_elements = second_gdf[second_gdf['Frame'] == (ti_frame + 1)]
        
        titj = geopandas.sjoin_nearest(ti_element, same_frame_elements,
                                       distance_col="pairwise_dist",
                                       max_distance=max_dist)
        chosen_closest_dist = titj.pairwise_dist.min()
        if (isnan(chosen_closest_dist)):
            print(f"This element has no neighbor within {max_dist}.")
        else:
            chosen_closest_cell = titj.pairwise_dist == chosen_closest_dist
            chosen_closest_row = titj[chosen_closest_cell]
            pairwise_tmp = pandas.concat([pairwise_elements, chosen_closest_row])
            pairwise_elements = pairwise_tmp
    return pairwise_elements

def merge_trackmate_rois(bact, directory_path):
    '''
    This function seeks to assign bacterial ROIs to tracks by merging the results of tracking (via Trackmate) and ROI detection (via Thresholding in ImageJ). It matches ROIs to spots detected in Trackmate based on the nearest centroid.
    For each cell, the function saves a CSV file of tracks and their corresponding ROIs in a folder called grouped.
    '''
    file_pattern = os.path.join(bact, "*.tif")
    file_list = glob.glob(file_pattern)
    out = bact+"grouped/"
    if not os.path.exists(out):
        os.makedirs(out)
    for file_path in file_list:
        if not file_path.endswith("overlay.tif"):
            filename = os.path.basename(file_path)
            filename2 = os.path.splitext(filename)[0]
            first = Path(f"{directory_path}/Single_Cell_Tracking_Output/{filename}_exportTracks.csv").as_posix()
            second = Path(f"{bact}/{filename2}.csv").as_posix()
            pairwise = xref_locations(first, second, 
                                  first_x='POSITION_X', 
                                  first_y='POSITION_Y', 
                                  first_z='POSITION_Z', 
                                  second_x='X', 
                                  second_y='Y', 
                                  second_z='Slice', 
                                  verbose=False)
            grouped = pairwise.groupby(pairwise.columns[2])[pairwise.columns[13]].apply(list).reset_index()
            grouped.rename(columns={grouped.columns[1]: 'object_ID_list'}, inplace=True)
            final_csv = Path(f"{out}/{filename2}_grouped.csv")
            grouped.to_csv(final_csv)
            final_csv2 = Path(f"{out}/{filename2}_pairwise.csv")
            pairwise.to_csv(final_csv2)
    return out

def apply_bact_overlays(bact, grouped):
    '''
    This function allows for visual quality-control of the bacterial-tracking step by adding colored overlays to the bacterial tracks in each single-cell video. Bacterial clusters of the same track remain the same color from frame to frame.
    All of the overlayed videos are saved in the folder bact with the title [cell_number]_overlay.tif.
    '''
    file_pattern = os.path.join(bact, "*.tif")
    file_list = glob.glob(file_pattern)
    colors = ["blue", "cyan", "green", "magenta", "orange", "red", "yellow"]
    for file_path in file_list: # cycle through images, open them and open the matching "grouped' csv file
        if not file_path.endswith("overlay.tif"):
            imp = ij.IJ.openImage(file_path)
            imp.show()
            f_name = os.path.basename(file_path)
            f_name = os.path.splitext(f_name)[0]
            input_csv = Path(f"{grouped}/{f_name}_grouped.csv")
            df = pandas.read_csv(input_csv)
            pouet = df['object_ID_list']
            rm.runCommand("Reset")
            input_ROI = Path(f"{bact}/{f_name}.zip")
            rm.open(f"{input_ROI}")
            for i in range(len(pouet)):
                single_row = df.iloc[i]
                random_color = random.choice(colors)
                row_cellids = single_row.object_ID_list
                row_cleaned = row_cellids.strip('[').strip(']')
                row_array = row_cleaned.split(', ')
                for cell in row_array:
                    cell_index = int(cell)
                    roi = rm.select(cell_index-1)
                    overlay_command = f"Overlay.addSelection('{random_color}',2);"
                    ij.py.run_macro(overlay_command)
            ij.py.run_macro("setMinAndMax(0, 100);")
            ij.py.run_macro("run('Flatten', 'stack');")
            method = 'max all'
            z_projector_result = ZProjector.run(imp, method)
            z_collapsed_image = ij.py.from_java(z_projector_result)
            z_collapsed_dataset = ij.py.to_dataset(z_projector_result)
            result_path = os.path.splitext(file_path)[0] + "_overlay.tif"
            if os.path.exists(result_path):
                os.remove(result_path)
            saved = ij.io().save(z_collapsed_dataset, result_path)       
            ij.py.run_macro("close();")
            rm.runCommand("Reset")

def measure_signals(cois, directory_path, grouped, bact, other):
    '''
    This function measures the amount of signal within each bacterial ROI for each single-cell video in bact. The signals to be measured are specified in the input list cois (short for channels of interest).
    The function saves the results as CSV files for each channel and cell in directory_path/measurement.
    '''
    file_pattern = os.path.join(other, "*.tif")
    file_list = glob.glob(file_pattern)
    out = directory_path+"/signal_measurement"
    if not os.path.exists(out):
        os.makedirs(out)
    rm.runCommand("Reset")
    #set measurement 
    set_string = f'Set Measurements...'
    measure_string = f'mean stack redirect=None decimal=2'
    ij.IJ.run(set_string, measure_string)
    prefix_bact = 'bact'
    # preparation of variables for file name call automation
    for file_path in file_list:
        f_name = os.path.basename(file_path)
        basename = os.path.splitext(f_name)[0]
        if basename.startswith(cois[0]):
            corename = basename.split("_", maxsplit=1)[1]
            input_csv = Path(f"{grouped}{corename}_grouped.csv")
            input_roi = Path(f"{bact}{corename}.zip")
            df0 = pandas.read_csv(input_csv)
            rm.open(f"{input_roi}")
            for channel in cois:
                c_path = Path(f"{other}{channel}_{corename}.tif").as_posix()
                image_c = ij.io().open(c_path)
                ij.ui().show(image_c)
                for n in range(len(df0)):
                    single_row = df0['object_ID_list'][n] #testing.iloc[10] #row number -2
                    row_cleaned = single_row.strip('[').strip(']')
                    terms = row_cleaned.split(", ")
                    nums = []
                    for term in terms:
                        nums.append(int(term))
                    nums.sort()
                    for bacteria in nums:
                        roi = rm.select(bacteria)
                        ij.IJ.run('Measure')
                output = Path(f"{out}/{channel}_{corename}.csv").as_posix()
                saving = ij.IJ.saveAs("Results", output)
                ij.IJ.run("Clear Results")
                ij.IJ.selectWindow(f"{channel}_{corename}.tif")
                ij.IJ.run('Close')
            rm.runCommand("Reset")
    return out

def combine_measurements(measurements, cois, directory_path):
    '''
    This function joins the separate CSV files of measurements for different channels into one single CSV file for each cell. It saves the output in a folder called final_output.
    The function also adds a track_id column to the CSV files to make it easier to distinguish between different tracks of bacteria.
    '''
    cell_list = []
    out = directory_path +"/final_output/"
    if not os.path.exists(out):
            os.makedirs(out)
    for filename in os.listdir(measurements):
        if filename.startswith(cois[0]):
            basename = filename[len(cois[0])+1:]
            corename = os.path.splitext(basename)[0]
            cell_list.append(corename)
    for cell in cell_list:
        dfs = []
        for i in range(len(cois)):
            coi = cois[i]
            input_csv = Path(f"{measurements}/{coi}_{cell}.csv")
            df = pandas.read_csv(input_csv)
            df.rename(columns={'Mean':f"Mean_{coi}"}, inplace=True)
            if i == 0:
                df = df.drop(columns = [df.columns[0], df.columns[2]], axis=1)
            else:
                df = df.drop(columns = [df.columns[0], df.columns[2], df.columns[3]], axis=1)
            dfs.append(df)
        final_results = pandas.concat(dfs, axis=1).iloc[:, [1,0,2]]
        input_csv2 = Path(f"{directory_path}/bact/grouped/{cell}_grouped.csv")
        df0 = pandas.read_csv(input_csv2)
        track_ids = []
        for index, row in df0.iterrows():
            cellnums = row['object_ID_list'].strip('[').strip(']').split(", ")
            for cellnum in cellnums:
                track_ids.append(index)
        final_results['track_id'] = track_ids
        output_path = Path(f"{out}/{cell}_final_result.csv")
        final_results.to_csv(output_path, index=False)
    return out

def get_cell_features(directory_path, distance, known, unit, cells_desired, features=f'area centroid stack'):
    '''
    This function obtains measurements for cells listed in cells_desired, including the area and centroid of each cell at each time step. 
    The measurements to be obtained are specified in the variable features, which should be a string with no spaces; e.g. 'area centroid stack'
    The function saves a separate CSV file with the measurements for each cell in the misc directory.
    '''
    image_path = Path(f"{directory_path}/channels.tif").as_posix()
    imp = ij.IJ.openImage(image_path)
    ij.ui().show(imp)
    rm.runCommand("Reset")
    add_rois(directory_path, sorted_tm_tracks_csv, roiset)
    ij.py.run_macro(f'run("Set Scale...", "distance={distance} known={known} unit={unit}");')    
    set_string = f'Set Measurements...'
    measure_string = features + f' redirect=None decimal=2'
    ij.IJ.run(set_string, measure_string)
    w = scyjava.jimport('ij.WindowManager')
    df = pandas.read_csv(tracks_csv, header=None)
    for cell_desired in cells_desired:
        nums = []
        if isinstance(cell_desired, int):
            row = df.iloc[cell_desired-1]
            nums = row[0].split(", ")
        elif isinstance(cell_desired, list):
            for c in cell_desired:
                row = df.iloc[c-1]
                nums += row[0].split(", ")
        arr = []
        for x in nums:
            arr.append(int(x))
        for i in range(len(arr)):
            cell = arr[i]
            roi = rm.getRoi(cell-1)
            roi2 = rm.select(cell-1)
            ij.IJ.run('Measure')
        misc_path = directory_path+'misc/'
        if not os.path.exists(misc_path):
            os.makedirs(misc_path)        
        output = Path(f"{misc_path}cell{cell_desired}_numbers.csv").as_posix()
        saving = ij.IJ.saveAs("Results", output)
        ij.IJ.run("Clear Results")
    ij.py.run_macro('close("*");')
    ij.IJ.selectWindow("Results")
    ij.IJ.run("Close")
    return misc_path

def get_cell_speed(misc_path, time_step):
    '''
    This function goes through all the 'numbers' CSV files in the misc directory and fills in columns for the distance traveled and speed of the cell at each time step.
    '''
    # goes through all the CSV files in misc and adds a distance and speed column
    file_pattern = os.path.join(directory_path+"misc/", "*.csv")
    file_list = glob.glob(file_pattern)
    for file_path in file_list:
        if file_path.endswith('numbers.csv'):
            df = pandas.read_csv(file_path)
            dists = [0]
            speeds = [0]
            for i in range(1, len(df)):
                currx = df.at[i, "X"]
                curry = df.at[i, "Y"]
                prevx = df.at[i-1, "X"]
                prevy = df.at[i-1, "Y"]
                dist = math.sqrt((currx-prevx)**2 + (curry-prevy)**2)
                speed = dist / time_step
                dists.append(dist)
                speeds.append(speed)
            df['Distance Traveled'] = dists
            df['Speed'] = speeds
            df.to_csv(file_path, index=False)
    return misc_path

def get_cell_volume(directory_path, z_step):
    '''
    This function iterates over the CSV files in the bact directory and calculates the bacterial volume of each cell frame-wise. 
    It puts the volume data in a new CSV file for each cell and saves them in the misc directory.
    '''
    file_pattern = os.path.join(directory_path+"bact/", "*.csv")
    file_list = glob.glob(file_pattern)
    out = directory_path+"misc/"
    if not os.path.exists(out):
        os.makedirs(out)
    f = pandas.read_csv(directory_path+"da_Frames.csv")
    frames_dict = pandas.Series(f['Total Frames'].values, index=f['Cell Number']).to_dict()
    for file_path in file_list:
        f_name = os.path.basename(file_path)
        cell_name = f_name[:len(f_name)-4]
        cell_number = cell_name.split("_")[1]
        df = pandas.read_csv(file_path)
        dict = {}
        curr_frame = 1
        total_area = 0
        total_frames = frames_dict[cell_number]
        for i in range(len(df)):
            if df.at[i, "Frame"] == curr_frame:
                total_area += df.at[i, "Area"]
            else:
                total_area = total_area * z_step
                dict.update({curr_frame : total_area})
                next_frame = curr_frame + 1
                if (i == len(df) - 1):
                    for skipped in range(next_frame, total_frames+1):
                        dict.update({skipped : 0})
                elif (df.at[i, "Frame"] != next_frame):
                    for skipped in range(next_frame, df.at[i, "Frame"]):
                        dict.update({skipped : 0})
                curr_frame = df.at[i, "Frame"]
                total_area = df.at[i, "Area"]
        data = pandas.DataFrame(list(dict.items()))
        data.columns = ['Frame', 'Total Volume']
        data.to_csv(out+cell_name+"_volume.csv", index=False)
    return out

def separate_slices(raw_image, ij,
                    wanted_x=True, wanted_y=True, wanted_z=10,
                    wanted_channel=2, cpus=8, overwrite=False, verbose=True):
    """Slice an image in preparation for cellpose.

    Eventually this should be smart enough to handle arbitrary
    x,y,z,channels,times as well as able to use multiple cpus for
    saving the data.  In its current implementation, it only saves 1
    z, 1 channel for every frame of an image into a series of files in
    its output directory.
    """

    input_base = os.path.basename(raw_image)
    input_dir = os.path.dirname(input_file)
    input_name = os.path.splitext(input_base)[0]
    output_directory = Path(f"{input_dir}/outputs/{input_name}_z{wanted_z}").as_posix()
    slice_directory = Path(f"{output_directory}/slices").as_posix()
    os.makedirs(output_directory, exist_ok=True)
    os.makedirs(slice_directory, exist_ok=True)
    if (raw_image is None):
        print("Starting to open the input file, this takes a moment.")
        raw_image = ij.io().open(input_file)
    if verbose:
        print(f"Opened input file, writing images to {output_directory}")

    data_info = {}
    for element in range(len(raw_image.dims)):
        name = raw_image.dims[element]
        data_info[name] = raw_image.shape[element]
    if verbose:
        print(
            f"This dataset has dimensions: X:{data_info['X']}",
            f"Y:{data_info['Y']} Z:{data_info['Z']} Time:{data_info['Time']}",
        )

    slices = []
    for timepoint in range(data_info['Time']):
        wanted_slice = raw_image[:, :, wanted_channel, wanted_z, timepoint]
        slice_data = ij.py.to_dataset(wanted_slice)
        output_filename = Path(f"{output_directory}/slices/frame_{timepoint}.tif").as_posix()
        if os.path.exists(output_filename):
            if overwrite:
                print(f"Rewriting {output_filename}")
                os.remove(output_filename)
                saved = ij.io().save(slice_data, output_filename)
            else:
                if verbose:
                    print(f"Skipping {output_filename}, it already exists.")
        else:
            saved = ij.io().save(slice_data, output_filename)
            if verbose:
                print(f"Saving image {input_name}_{timepoint}.")
        slices.append(wanted_slice)
    print(f"Returning the output directory: {output_directory}")
    return raw_image, slices, output_directory

def get_total_fluorescence(directory_path, sum_channels, imp, cells_desired, channel_dict, target_channel):
    # target channel should be a string ex. LC3
    imp = ij.IJ.openImage(sum_channels)
    imp.show()
    imp.setC(channel_dict[target_channel])
    set_string = f'Set Measurements...'
    measure_string = f'mean integrated stack redirect=None decimal=2'
    ij.IJ.run(set_string, measure_string)
    tracks_csv = directory_path+'da_Tracks.csv'
    df = pandas.read_csv(tracks_csv, header=None)
    for cell_desired in cells_desired:
        nums = []
        if isinstance(cell_desired, int):
            row = df.iloc[cell_desired-1]
            nums = row[0].split(", ")
        elif isinstance(cell_desired, list):
            for c in cell_desired:
                row = df.iloc[c-1]
                nums += row[0].split(", ")
        arr = []
        for x in nums:
            arr.append(int(x))
        for i in range(len(arr)):
            cell = arr[i]
            roi = rm.getRoi(cell-1)
            roi2 = rm.select(cell-1)
            ij.IJ.run('Measure')    
        output = Path(f"{directory_path}/misc/cell{cell_desired}_{target_channel}_fluo.csv").as_posix()
        saving = ij.IJ.saveAs("Results", output)
        ij.IJ.run("Clear Results")
    ij.py.run_macro('close("*");')
    return output

# Execution

## step 1

### Open Raw Image

In [4]:
# the directory_path is the path to your main directory containing your time lapse video. If there is multiple time-lapses, field of views or repeats
# it is required to create subdirectory containing only one of the video, and 'directory_path' should be the path to this subdirectory.
directory_path = "/home/saka/Documents/Lab_stuff/confocal/new_video_hmdm/scene1/"
raw_image_path = directory_path+"scene1.tif" # replace the 'scene1 by the name of your file'

In [5]:
# Opens the raw image
raw_image = open_raw_image(raw_image_path)

Custom TB Handler failed, unregistering
Traceback (most recent call last):
  File "DefaultIOService.java", line 86, in org.scijava.io.DefaultIOService.open
Exception: Java Exception

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/saka/sw/local/fiji/2023/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_24982/2963155440.py", line 2, in <module>
    raw_image = open_raw_image(raw_image_path)
  File "/tmp/ipykernel_24982/2620952026.py", line 5, in open_raw_image
    raw_image = ij.io().open(raw_image_path)
java.lang.java.lang.IndexOutOfBoundsException: java.lang.IndexOutOfBoundsException: Index: 1, Size: 1

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/saka/sw/local/fiji/2023/lib/python3.10/site-packages/IPython/core/interactiveshell.p

In [ ]:
# if you need to train Cellpose, you'll need a folder containing individual images of the channel of interest.
# To get those, run this:
separate_slices(raw_image_path, ij, raw_image=None,
                    wanted_x=True, wanted_y=True, wanted_z=10,
                    wanted_channel=2, overwrite=False, verbose=True)

### Process Images
This is to create and save 2 types of images, a single slice image from the raw image used for the segmentation and tracking using Trackmate-cellpose.
The other one is a z-projection max of the raw image used for the quality control of the tracking and the selection of cells of interest.

In [ ]:
# enter the slice number used for the training of cellpose model, or the one that gives a good visual contrast of the cell
wanted_z = 10

# method for the z-projection, the addition of 'all' makes sure it will be done on the whole time-lapse
method = 'max all'

# Here, enter the dimensions of the projection. The numbers in the brackets corresponds to [channels, slices, frames]. 
# the number of each of these must be entered to make sure that during the save of the file, the dimensions are in the proper order.
# Change the channels number and frame number to your own image dimensions. 
dimensions = [3, 1, 121]

#
get_sum_channels = True

In [ ]:
# run the function
channels_image, max_channels = process_images(directory_path, raw_image, method, wanted_z, dimensions, get_sum_channels)

### Process Channels
This is to split the raw image into single channel images. The bacteria channel is used for the subsequent bacteria tracking while the other channels for fluorescence quantification purposes

In [5]:
# Enter the names of the channels of interest and their number in the stack. Reminder that in python, the first channel is the number '0' while in fiji it is '1'
channel_dict = { 
    'LV' : 1,
    'bact' : 0,
    'TPMT' : 2
}
# As above, enter the dimensions of the future single channel z-stacks timelapse. The numbers in the brackets corresponds to [channels, slices, frames]. 
# Change the slice number and frame number to your own image dimensions.
dimensions = [1, 11, 121]

In [ ]:
# run the functions
process_channels(directory_path, raw_image, channel_dict, dimensions)

## step 2

### Track Cells
The segmentation and tracking is done using Trackmate-cellpose. The cell below is listing the different parameter used for the detection of the cells with cellpose (dsettings) and the tracking paramters using the LAP tracker.

In [6]:
dsettings = { # specify parameters for the detection step in Trackmate (Threshold Detector)
    'TARGET_CHANNEL' : ij.py.to_java(0),
    'SIMPLIFY_CONTOURS' : True,
    'CELLPOSE_MODEL' : PretrainedModel.CUSTOM,
    'CELLPOSE_MODEL_FILEPATH' : "/home/saka/Documents/CP_20220523_104016",
    'USE_GPU' : True,
    'OPTIONAL_CHANNEL_2' : ij.py.to_java(0),
    'CELL_DIAMETER' : 32,
    'CELLPOSE_PYTHON_FILEPATH' : "/home/saka/sw/local/fiji/2023/bin/python3.10",
}
tsettings = { # specify parameters for the tracking step in Trackmate (LAP Tracker)
    'LINKING_MAX_DISTANCE' : 15.0,
    'ALLOW_GAP_CLOSING' : True,
    'GAP_CLOSING_MAX_DISTANCE' : 15.0,
    'MAX_FRAME_GAP' : ij.py.to_java(2),
    'ALLOW_TRACK_SPLITTING' : False,
    'SPLITTING_MAX_DISTANCE' : 15.0,
    'ALLOW_TRACK_MERGING' : False,
    'ALTERNATIVE_LINKING_COST_FACTOR' : 1.05,
    'MERGING_MAX_DISTANCE' : 15.0,
    'CUTOFF_PERCENTILE' : 0.9,
}

# These are the parameters to fill when running the "set scale..." in fiji. This is crucial to make sure the distances during the tracking stay coherent along the workflow
distance = 2048 
known = 319.45
unit = "µm"

In [ ]:
# run the segmentation and tracking:
tm_tracks_csv, model, imp = track_cells(channels_image, directory_path, dsettings, tsettings, distance, known, unit)
# This function extract the ROIs from the trackmate results, and saves them in a directory named 'cell_rois'
roiset = get_cell_rois(directory_path, model, imp)

## step 3

### Process Trackmate Output
The step is to first sort the trackmate output table in chronological order for each tracks and save the modified version. The ROIs from trackmate results are exported and saved in the cell_rois directory created in the process.

In [ ]:
# this sorts the tracks in the 'exportTracks.csv' file
sorted_tm_tracks_csv = sort_trackmate_export(tm_tracks_csv, directory_path)

### Create CSV File of Tracks

In [ ]:
# for each tracks, this extract the index+1 of each spot to put 
tracks_csv = create_tracks_csv(sorted_tm_tracks_csv, directory_path)

### Add Ordered ROIs to ROI Manager

In [8]:
# This function adds the sorted ROIs of each cell in each tracks in the ROI manager
add_rois(directory_path, sorted_tm_tracks_csv, roiset)

## Step 4
This step is for the quality control of the tracking result. The 'max_channels' image generated in step 1 is opened and the ROIs of each cells and each tracks are overlayed. 

### Apply Colored Overlays

In [9]:
# This opens the z-projection max saved in step 1 and overlay the ROIs of each track with a randomly picked color
apply_cell_overlays(directory_path, max_channels, tracks_csv, sorted_tm_tracks_csv, roiset)

In [ ]:
# here enters the cell number which match the line in the da_tracks.csv file that matches the label of the cells at the first frame or start of the track. 
cells_desired = [4, 10, 13, 49, 53, 67, 78, 107, [17,181]]

## step 5 

### Duplicate Cells of Interest
This function allows for the duplication of the single cell signal from the list of cell given in the 'cells_desired' list given above. The duplication is reiterated for each channel listed in the 'channel_dict' dictionnary early in the workflow. The images are saved in the 'bact' directory for the bacterial channel, and in the 'other_channels' directory for the other ones.

In [ ]:
bact, other = isolate_single_cell(directory_path, channel_dict, tracks_csv, sorted_tm_tracks_csv, roiset, cells_desired)

## step 6
This step is the bacterial tracking workflow. First the ROIs around the bacteria are generated by thresholding followed by analyze particles function of fiji. The XYZ coordinate of the ROIs are also collected, and the process is reiterated for each single cells in the 'bact' directory.
Then tracking is done using Trackmate using the threshold detector and the LAP tracker and the tracks table for each cells is saved. Finally, the fiji output and the trackmate output are merged to assign the closest ROI to the centroid of the spots in detected in trackmate to have only one ROI per bacteria / phagosome.

### Get Bacterial ROIs

In [ ]:
# enter the desired threshold values that will be applied to all the images for the collection of bacteria ROIs. The high threshold value is usually the maximum value of the image, which is 255 for an 8-bit image.
# The size is expressed in square micrometer.
threshold_low = 15
threshold_high = 255
threshold_size = 0.5

In [ ]:
get_bacteria_rois(bact, directory_path, threshold_low, threshold_high, threshold_size, distance, known, unit)

### Track Bacteria
This is the tracking step in Trackmate

In [ ]:
dsettings = { # specify parameters for the detection step in Trackmate (Threshold Detector)
    'TARGET_CHANNEL' : ij.py.to_java(1),
    'SIMPLIFY_CONTOURS' : False,
    'INTENSITY_THRESHOLD' : 15.0,
}
quality_filter = 45
tsettings = { # specify parameters for the tracking step in Trackmate (LAP Tracker)
    'LINKING_MAX_DISTANCE' : 2.0,
    'ALLOW_GAP_CLOSING' : True,
    'GAP_CLOSING_MAX_DISTANCE' : 9.0,
    'MAX_FRAME_GAP' : ij.py.to_java(2),
    'ALLOW_TRACK_SPLITTING' : False,
    'SPLITTING_MAX_DISTANCE' : 15.0,
    'ALLOW_TRACK_MERGING' : False,
}

In [ ]:
track_bacteria(bact, directory_path, dsettings, quality_filter, tsettings, distance, known, unit, display=False)

### Merge Trackmate Output with Fiji ROIs

In [ ]:
grouped = merge_trackmate_rois(bact, directory_path)

### Visualize Bacterial Tracking
this is the quality control step, the ROIs selected for each spot of the bacterial tracks are overlayed on the 'bact_channel' image of its corresponding single cell. the overlay is then flatten and the image is transformed in a z-projection 'max', and saved with a suffix "_overlay" to it in the 'bact' directory.

In [ ]:
apply_bact_overlays(bact, grouped)

## Step 7a - single cell quantification

### Obtain Cell Features
Here the objective is to measure and collect various outputs, e.g. cell velocity and bacterial volume in the way of the BBQ method. 

In [ ]:
# for cell velocity of speed calculation, enter the time between the frames
time_step = 5
misc_path = get_cell_features(directory_path, distance, known, unit, cells_desired) #this is to get area or other metrics that fiji can measure

In [ ]:
get_cell_speed(misc_path, time_step) # this is to calculate cells velocity and add this result to the csv file generated above

In [ ]:
z_step = 1 # enter the distance between the slices in the z-stack
get_cell_volume(directory_path, z_step)

### get total fluorescence per cell
this step would open the sum_channel generated in step1 and use it to measure the total fluorescence intensity over time

In [ ]:

sum_channels = f"{directory_path}/sum_channels.tif"
if not os.path.exists(sum_channels):
    raw_image = open_raw_image(raw_image_path)
    imp = ij.py.to_imageplus(raw_image)
    get_z_stack(directory_path, imp, 'sum all', dimensions)
    
target_channel = 'LV' #enter the name of the channel

In [ ]:
get_total_fluorescence(directory_path, sum_channels, cells_desired, channel_dict, target_channel)

## Step 7b - single phagosome quantification
For the step, enter the name of the channel of interests in the list of cois below. The name must match the one given for the 'channel_dict' at the beginning.
then run the 'measure_signal' function to measure the mean fluorescence intensity in the ROIs of each single bacteria in the single cells

### Take Measurements

In [ ]:
cois = ['LV'] # channels of interest

In [ ]:
measurements = measure_signals(cois, directory_path, grouped, bact, other)

### Combine Measurements into Single CSV File

In [ ]:
# For multiple channels, this allow the concatenation of the measurements in multiples channels in a single csv file
combine_measurements(measurements, cois, directory_path)

# Variable Values / Locations

### User-Set Variables

In [ ]:
directory_path = "/Users/volkenlab/Documents/Anushka/20240229 hmdm h37dsred lv/scene1/"
raw_image_path = directory_path+"Experiment-2099-Split Scenes-01.czi"
wanted_z = 11
method = 'max all'
dimensions = [3, 1, 121]
channel_dict = {
    'LC3' : 0, 
    'LV' : 2, 
    'bact' : 1
}
cois = ['LC3', 'LV']
threshold_low = 30
threshold_high = 255
threshold_size = 0
distance = 2048
known = 319.45
unit = "µm"
time_step = 5
distance_between_slices = 1

### Computer-Generated Variables

In [7]:
channels_image = directory_path+'channels.tif'
max_channels = directory_path+'max_channels.tif'
tracks_csv = directory_path+'da_Tracks.csv'
sorted_tm_tracks_csv = directory_path+'exportTracks_edited.csv'
tm_tracks_csv = directory_path+'exportTracks.csv'
roiset = directory_path+'cell_rois/'
bact = directory_path+'bact/'
other = directory_path+'other_signals/'
grouped = directory_path+'bact/grouped/'
measurements = directory_path+'signal_measurement/'
misc_path = directory_path+'misc/'

## other stuff

In [ ]:
lv_path = f"{directory_path}channel_bact.tif"
imp = ij.IJ.openImage(lv_path)
zproject_sum = ZProjector.run(imp, "sum all")
sum_channels = directory_path + "sum_channel_bact.tif"
ij.IJ.saveAs(zproject_sum, "Tiff", ij.py.to_java(sum_channels))

In [ ]:
sum_channels_path = f"{directory_path}sum_channel.tif"
imp = ij.IJ.openImage(sum_channels_path)
imp.show()
imp.setC(2)
set_string = f'Set Measurements...'
measure_string = f'mean integrated stack redirect=None decimal=2'
ij.IJ.run(set_string, measure_string)
cells_desired = [4, 10, 13, 49, 53, 67, 78, 107, [17, 181]]
df = pandas.read_csv(tracks_csv, header=None)
for cell_desired in cells_desired:
    nums = []
    if isinstance(cell_desired, int):
        row = df.iloc[cell_desired-1]
        nums = row[0].split(", ")
    elif isinstance(cell_desired, list):
        for c in cell_desired:
            row = df.iloc[c-1]
            nums += row[0].split(", ")
    arr = []
    for x in nums:
        arr.append(int(x))
    for i in range(len(arr)):
        cell = arr[i]
        roi = rm.getRoi(cell-1)
        roi2 = rm.select(cell-1)
        ij.IJ.run('Measure')    
    output = Path(f"{misc_path}cell{cell_desired}_LVfluo.csv").as_posix()
    saving = ij.IJ.saveAs("Results", output)
    ij.IJ.run("Clear Results")
ij.py.run_macro('close("*");')

In [ ]:
# This is to make the number of each lane of the "_grouped csv file" to be usable for the ROI manager later
image_path = f"{other}/LV_cell_4.tif"
imp = ij.IJ.openImage(image_path)
imp.show()
grouped_path = Path(f"{grouped}/cell_4_grouped.csv")
df = pandas.read_csv(grouped_path)
pouet = df['object_ID_list']
input_ROI = Path(f"{bact}/cell_4.zip")
rm.open(f"{input_ROI}")
for i in range(len(pouet)):
    single_row = df.iloc[i]
    row_cellids = single_row.object_ID_list
    row_cleaned = row_cellids.strip('[').strip(']')
    row_array = row_cleaned.split(', ')
    for cell in row_array:
        cell_index = int(cell)
        roi = rm.select(cell_index)
        overlay_command = f"Overlay.addSelection('Magenta', 2);"
        ij.py.run_macro(overlay_command)
        

In [ ]:
# opening of the bacterial channel and the Fiji ROIs
bact_channel = f"{frame_path}/channel_bact.tif"
image = ij.io().open(bact_channel)
shown = ij.ui().show(image)
roi_input = f"{frame_path}/channel_bact.zip"
rm.open(roi_input)

In [ ]:
# For each line of the grouped CSV file, a color is randomly picked and the ROIs of the lines are overlayed on the image with the color. 
import random
colors = ["blue", "cyan", "green", "magenta", "orange", "red", "white", "yellow"]
for i in ROI_ID:
    random_color = random.choice(colors)
    rm.select(i)
    overlay_command = f"Overlay.addSelection('{random_color}', 2);"
    ij.py.run_macro(overlay_command)
    